In [74]:
import pandas as pd
import numpy as np

In [75]:
full=pd.merge(pd.read_csv("family_data.csv"),pd.read_csv("sample_submission.csv"))

In [76]:
def prefCost(people,choice=-1):
    consolation=[[0,0],[50,0],[50,9],[100,9],[200,9],[200,18],[300,18],[300,36],[400,36],[500,36],[500,36+398]]
    return (consolation[choice][0]+consolation[choice][1]*people) 

def accCost(currDay):
    segregated=full.groupby(by=['assigned_day']).sum()
    #For case no. 100, currDay=prevDay
    segregated.loc[101]=segregated.loc[100]

def fullCost(full):
    total=0
    for day in range(len(segregated)):
        total+=prefCost(full.iloc[day]['n_people'],full.iloc[day]['assigned_choice'])

In [37]:
np.random.seed(1)
full['assigned_choice']=pd.Series(np.random.randint(0,10,size=5000))

In [38]:
#full['assigned_day']=full['choice_'+str(full['assigned_choice'])]
for i in range(5000):
    full['assigned_day'][i]=full['choice_'+str(full['assigned_choice'][i]).rstrip()][i]
    full['pref_cost'][i]=prefCost(full['n_people'][i],full['assigned_choice'][i])
full.head()

C:\Users\GurpreetSingh\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GurpreetSingh\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people,assigned_day,assigned_choice,pref_cost
0,0,52,38,12,82,33,75,64,76,10,28,4,75,5,272
1,1,26,4,82,5,11,47,38,6,66,61,4,66,8,544
2,2,100,54,25,12,27,82,10,89,80,33,3,33,9,608
3,3,2,95,1,96,32,6,40,31,9,59,2,6,5,236
4,4,53,1,47,93,26,3,46,16,42,39,4,53,0,0


In [154]:
#Assign -1 to all in full
full['assigned_choice']=-1
full['assigned_day']=-1
full['pref_cost']=-1

#Make a table for day-wise grouping
day_table=pd.DataFrame({'Day': range(1,101),'Count':0})
day_table.set_index('Day')


family_id_range=np.arange(5000)
print(family_id_range)

[   0    1    2 ... 4997 4998 4999]


In [155]:
for day in range(0,100):
    print("Working for day "+str(day+1))
    print("=========================")
    day_table.loc[day]['Count']=0
    for family in range(5000):
        #rint("Working on "+str(family))
        if(day_table.loc[day]['Count']>=205):
            print("Total for day "+str(day+1)+"="+str(day_table.loc[day]['Count']))
            break
        if(full['assigned_day'][family]>-1):
            continue
        full['assigned_day'][family]=day+1
        day_table['Count'][day]+=full['n_people'][family]
        #print("Adding family_id "+str(family))
        #for choice in range(0,5):
        #    if(full['choice_'+str(choice)][family]==day):
        #        full['assigned_choice'][family]=choice
        #        day_table['Count'][day]+=full['n_people'][family]
        #        family_id_range=np.delete(family_id_range,[family])
        #        break

Working for day 1
Total for day 1=205
Working for day 2
Total for day 2=207
Working for day 3
Total for day 3=207
Working for day 4
Total for day 4=210
Working for day 5
Total for day 5=205
Working for day 6
Total for day 6=207
Working for day 7
Total for day 7=207
Working for day 8
Total for day 8=206
Working for day 9
Total for day 9=208
Working for day 10
Total for day 10=206
Working for day 11
Total for day 11=208
Working for day 12
Total for day 12=209
Working for day 13
Total for day 13=205
Working for day 14
Total for day 14=209
Working for day 15
Total for day 15=205
Working for day 16
Total for day 16=207
Working for day 17
Total for day 17=207
Working for day 18
Total for day 18=208
Working for day 19
Total for day 19=208
Working for day 20
Total for day 20=205
Working for day 21
Total for day 21=208
Working for day 22
Total for day 22=208
Working for day 23
Total for day 23=210
Working for day 24
Total for day 24=205
Working for day 25
Total for day 25=205
Working for day 26

In [153]:
day_table

,Day,Count
0,1,213
1,2,212
2,3,212
3,4,210
4,5,211
...,...,...
95,96,215
96,97,212
97,98,213
98,99,210


In [157]:
day_table['Count'].sum()
full

,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people,assigned_day,assigned_choice,pref_cost
0,0,52,38,12,82,33,75,64,76,10,28,4,1,-1,-1
1,1,26,4,82,5,11,47,38,6,66,61,4,1,-1,-1
2,2,100,54,25,12,27,82,10,89,80,33,3,1,-1,-1
3,3,2,95,1,96,32,6,40,31,9,59,2,1,-1,-1
4,4,53,1,47,93,26,3,46,16,42,39,4,1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,16,1,66,33,18,70,56,46,86,60,4,-1,-1,-1
4996,4996,88,66,20,17,26,54,81,91,59,48,2,-1,-1,-1
4997,4997,32,66,54,17,27,21,74,81,3,7,6,-1,-1,-1
4998,4998,67,92,4,17,53,77,1,12,26,70,5,-1,-1,-1


In [43]:
segregated=full.groupby(by=['assigned_day']).sum()
segregated.loc[101]=segregated.loc[100]
segregated[(segregated['n_people']<125)|(segregated['n_people']>300)]

,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people,assigned_choice,pref_cost
assigned_day,,,,,,,,,,,,,,
1,674569,10050,10542,10219,10863,10962,10721,9970,10540,11008,11636,1121,1009,61286
3,198317,2989,3635,3025,3316,3186,3515,2982,3745,3032,2915,336,379,24037
4,216164,3149,3839,3365,3568,3344,3276,3356,3433,3365,3616,361,414,25486
5,223105,3186,3515,3518,2758,3673,2999,3332,3314,3517,2878,393,386,24976
10,259444,3925,3664,3844,3937,4778,3926,4444,4226,4032,4633,419,432,27213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,32312,671,752,797,632,947,821,669,895,614,771,74,67,3895
98,45931,1172,805,1113,1126,915,711,976,989,1115,734,97,81,5130
99,38683,860,652,733,685,550,904,750,777,720,771,65,64,3981


In [36]:
#a=full.groupby('assigned_day').sum()
#a[(a['n_people']<125)|(a['n_people']>300)]
print(full['n_people'].sum())

21003


In [14]:
total_cost=0
for i in range(5000):
    full['pref_cost'][i]=prefCost(full['n_people'][i],full['assigned_choice'][i])
full.head()
    

,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people,assigned_day,assigned_choice,pref_cost
0,0,52,38,12,82,33,75,64,76,10,28,4,52,0,0
1,1,26,4,82,5,11,47,38,6,66,61,4,26,0,0
2,2,100,54,25,12,27,82,10,89,80,33,3,100,0,0
3,3,2,95,1,96,32,6,40,31,9,59,2,2,0,0
4,4,53,1,47,93,26,3,46,16,42,39,4,53,0,0
